### Predicting Recidivism Rates - Notebook 1.
In this notebook:
1. The two `.csvs are merged` from the AWS scrape.
2. The features related to the crime are created (not yet categorized).
3. Anyone having more than 30 years remaining on their projected release date has been filtered out of the dataset. 
4. The target column has been created. The convicted crime is still visible in another column, and then we have our y of `1 or 0`. 1 indicates the inmate reoffended, 0 indicates they did not.

In [25]:
import pandas as pd

In [26]:
pd.set_option('display.max_columns', 100)

In [27]:
#need to move this into the GA folder
inmate_details = pd.read_csv('../datasets/my_data/inmate_details_FINAL.csv')

In [28]:
#need to move this into the GA folder
priors = pd.read_csv('../datasets/my_data/priors_FINAL.csv')

In [29]:
inmate_details.shape

(56600, 11)

In [30]:
priors.shape

(47500, 19)

In [31]:
inmate_details.loc[44_199]

Unnamed: 0                         44199
name                 Sergio Lopez-Garcia
sex                                 Male
race                            Hispanic
age                                   30
max_sentence                    15 years
prison_unit                    Hightower
DOB                             7/6/1989
home_county                      Midland
TDCJ_ID                          1831464
proj_release_date              9/23/2027
Name: 44199, dtype: object

In [32]:
priors.loc[44_199]

Unnamed: 0                        44199
name                Sergio Lopez-Garcia
TDCJ_ID                         1831464
pr_crime_0           AGG SEX ASLT CHILD
pr_commit_date_0              9/23/2012
pr_term_0                      15 years
pr_begins_0                   9/23/2012
pr_crime_1                      No_data
pr_commit_date_1                No_data
pr_term_1                       No_data
pr_begins_1                     No_data
pr_crime_2                      No_data
pr_commit_date_2                No_data
pr_term_2                       No_data
pr_begins_2                     No_data
pr_crime_3                      No_data
pr_commit_date_3                No_data
pr_term_3                       No_data
pr_begins_3                     No_data
Name: 44199, dtype: object

In [33]:
priors[:2]

,Unnamed: 0,name,TDCJ_ID,pr_crime_0,pr_commit_date_0,pr_term_0,pr_begins_0,pr_crime_1,pr_commit_date_1,pr_term_1,pr_begins_1,pr_crime_2,pr_commit_date_2,pr_term_2,pr_begins_2,pr_crime_3,pr_commit_date_3,pr_term_3,pr_begins_3
0,0,Micah Troy Duncan,1753676,CAPITAL MURDER PERSON U/SIX YR,9/9/2010,Life Without Parole,9/15/2010,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data
1,1,Heyman Harold Guillory,1472234,CAPITAL MURDER,3/28/2007,Life Without Parole,3/29/2007,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data


In [34]:
inmate_details[:2]

,Unnamed: 0,name,sex,race,age,max_sentence,prison_unit,DOB,home_county,TDCJ_ID,proj_release_date
0,0,Micah Troy Duncan,Male,White,32,7000 years,Allred,7/22/1987,Grayson,1753676,1/1/5555
1,1,Heyman Harold Guillory,Male,Black,44,7000 years,Allred,7/29/1975,No_data,1472234,1/1/5555


In [35]:
inmate_details.isnull().sum()

Unnamed: 0            0
name                  0
sex                   0
race                  0
age                   0
max_sentence         48
prison_unit           0
DOB                   0
home_county           0
TDCJ_ID               0
proj_release_date     0
dtype: int64

In [36]:
#looking at 'pr_term_0' - del those nulls 
priors.isnull().sum()

Unnamed: 0            0
name                  0
TDCJ_ID               0
pr_crime_0            0
pr_commit_date_0      0
pr_term_0            63
pr_begins_0           0
pr_crime_1            3
pr_commit_date_1      0
pr_term_1           455
pr_begins_1           0
pr_crime_2            5
pr_commit_date_2      0
pr_term_2           443
pr_begins_2           0
pr_crime_3            1
pr_commit_date_3      0
pr_term_3           398
pr_begins_3           0
dtype: int64

In [37]:
df = pd.merge(priors, inmate_details, how= 'left') #need the smaller df first

In [38]:
df.shape

(47500, 27)

In [39]:
df['TDCJ_ID'].nunique()

47500

In [40]:
df = df.drop(columns = 'Unnamed: 0')

In [41]:
df.dropna(subset =['pr_term_0'], inplace=True)

In [42]:
df[:5]

,name,TDCJ_ID,pr_crime_0,pr_commit_date_0,pr_term_0,pr_begins_0,pr_crime_1,pr_commit_date_1,pr_term_1,pr_begins_1,pr_crime_2,pr_commit_date_2,pr_term_2,pr_begins_2,pr_crime_3,pr_commit_date_3,pr_term_3,pr_begins_3,sex,race,age,max_sentence,prison_unit,DOB,home_county,proj_release_date
0,Micah Troy Duncan,1753676,CAPITAL MURDER PERSON U/SIX YR,9/9/2010,Life Without Parole,9/15/2010,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,White,32,7000 years,Allred,7/22/1987,Grayson,1/1/5555
1,Heyman Harold Guillory,1472234,CAPITAL MURDER,3/28/2007,Life Without Parole,3/29/2007,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,Black,44,7000 years,Allred,7/29/1975,No_data,1/1/5555
2,"Batiste Joseph Breaux, Jr",1954769,CAPITAL MURDER,8/5/2008,Life Without Parole,8/14/2008,BURG HABIT,1/4/2002,12 years,7/3/2003,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,Black,36,7000 years,Allred,9/27/1982,Jefferson,1/1/5555
3,Glen Leon Dukes,2023028,CAPITAL MURDER,6/2/2012,Life Without Parole,2/7/2013,CONT TRAFF OF PERSONS,10/1/2011,Life,2/7/2013,CONT TRAFF OF PERSONS,10/1/2011,Life,2/7/2013,POSS C/S W/I DEL,11/8/2004,10 years,11/8/2004,Male,Black,50,7000 years,Allred,3/16/1969,Bexar,1/1/5555
4,Joshua Ray Juarez,1480997,CAPITAL MURDER,12/13/2006,Life Without Parole,12/15/2006,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,Hispanic,30,7000 years,Allred,4/13/1989,Harris,1/1/5555


In [43]:
#original merged datasets
df.to_csv('../datasets/my_data/complete_raw_df.csv', index=False)

Below getting the `proj_release_date` in a usable format to filter. For the actual model, only looking at inmates that have 30 years or fewer remaining on their sentence. 

Since some of the years are too far away (ie 5555), I have to create a function to get them to a reasonable spot, yet still clearly beyond the scope of what I would want for the model.

In [44]:
series = df['proj_release_date']

In [45]:
split_series = []
for term in series:
    split_term = term.split('/')
    
    split_series.append(split_term)

In [46]:
#getting rid of the crazy terms like '5555', '9999', and anything more than 130 years away
for term in split_series:
    term_int = int(term[2])
    
    if term_int > 2150:
        term[2] = '2150'

In [47]:
years = []
for term in split_series:
    years.append(term[2])

In [48]:
list_years = []
for term in years:
    int_term = int(term)
    
    list_years.append(int_term)

In [49]:
df['year_of_release'] = list_years

In [50]:
#getting everything at 2150 filtered out of the dataset
df = df[df['year_of_release'] != 2150]

In [51]:
df['proj_release_date'] = pd.to_datetime(df['proj_release_date'])

In [52]:
df.dtypes

name                         object
TDCJ_ID                       int64
pr_crime_0                   object
pr_commit_date_0             object
pr_term_0                    object
pr_begins_0                  object
pr_crime_1                   object
pr_commit_date_1             object
pr_term_1                    object
pr_begins_1                  object
pr_crime_2                   object
pr_commit_date_2             object
pr_term_2                    object
pr_begins_2                  object
pr_crime_3                   object
pr_commit_date_3             object
pr_term_3                    object
pr_begins_3                  object
sex                          object
race                         object
age                           int64
max_sentence                 object
prison_unit                  object
DOB                          object
home_county                  object
proj_release_date    datetime64[ns]
year_of_release               int64
dtype: object

In [53]:
df['proj_release_date'][:2]

322   2101-04-22
323   2113-10-19
Name: proj_release_date, dtype: datetime64[ns]

In [54]:
#checking the shape before doing a large filter on release dates
df.shape[0]

45148

In [55]:
#for the model, i only want to look at the inmates that have projected release in the next 30 years
df = df[df['proj_release_date'] <= '2049-8-31']

In [56]:
df.shape[0]

40828

In [57]:
#confirming the filter worked correctly
len(df[df['proj_release_date'] > '2049-9-01'])

0

In [58]:
df[:2]

,name,TDCJ_ID,pr_crime_0,pr_commit_date_0,pr_term_0,pr_begins_0,pr_crime_1,pr_commit_date_1,pr_term_1,pr_begins_1,pr_crime_2,pr_commit_date_2,pr_term_2,pr_begins_2,pr_crime_3,pr_commit_date_3,pr_term_3,pr_begins_3,sex,race,age,max_sentence,prison_unit,DOB,home_county,proj_release_date,year_of_release
590,Victor Cantu,327167,HARASSMENT BY PER/CORR FACIL,2/20/2003,9 years,2/20/2003,HARASSMENT BY PER/CORR FACIL,2/20/2003,9 years,2/20/2003,MURDER W/DEADLY WPN,3/3/1980,60 years,9/7/1980,No_data,No_data,No_data,No_data,Male,Hispanic,71,60 years,Allred,3/2/1948,Hockley,2044-11-23,2044
628,Pascual Gutierrez Salinas,394125,POSS DDLY WPM PENAL INSTI,7/19/1997,5 years,2/10/1998,MURDER,8/6/1984,60 years,8/6/1984,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,Hispanic,62,60 years,Allred,4/21/1957,Cameron,2047-08-16,2047


Below setting up the function and mapping each row to fill out `feature_crime` with the appropriate crime. This is the first crime the inmate committed (that's been scraped). 

In [59]:
df[df['pr_crime_0'].str.contains('No')]

,name,TDCJ_ID,pr_crime_0,pr_commit_date_0,pr_term_0,pr_begins_0,pr_crime_1,pr_commit_date_1,pr_term_1,pr_begins_1,pr_crime_2,pr_commit_date_2,pr_term_2,pr_begins_2,pr_crime_3,pr_commit_date_3,pr_term_3,pr_begins_3,sex,race,age,max_sentence,prison_unit,DOB,home_county,proj_release_date,year_of_release


In [60]:
df['pr_crime_0'].isnull().sum()

0

In [61]:
df['feature_crime'] = 'TBD'

In [62]:
df[['name', 'pr_crime_0', 'pr_crime_1', 'pr_crime_2', 'pr_crime_3', 'feature_crime']][:3]

,name,pr_crime_0,pr_crime_1,pr_crime_2,pr_crime_3,feature_crime
590,Victor Cantu,HARASSMENT BY PER/CORR FACIL,HARASSMENT BY PER/CORR FACIL,MURDER W/DEADLY WPN,No_data,TBD
628,Pascual Gutierrez Salinas,POSS DDLY WPM PENAL INSTI,MURDER,No_data,No_data,TBD
638,Larry Dewayne Manuel,MURDER,No_data,No_data,No_data,TBD


In [63]:
df = df.reset_index(drop=True)

In [64]:
df[['name', 'pr_crime_0', 'pr_crime_1', 'pr_crime_2', 'pr_crime_3', 'feature_crime']][:3]

,name,pr_crime_0,pr_crime_1,pr_crime_2,pr_crime_3,feature_crime
0,Victor Cantu,HARASSMENT BY PER/CORR FACIL,HARASSMENT BY PER/CORR FACIL,MURDER W/DEADLY WPN,No_data,TBD
1,Pascual Gutierrez Salinas,POSS DDLY WPM PENAL INSTI,MURDER,No_data,No_data,TBD
2,Larry Dewayne Manuel,MURDER,No_data,No_data,No_data,TBD


In [65]:
df.loc[3]

name                   Wayne Edward Cole
TDCJ_ID                           358223
pr_crime_0            BURG OF HABITATION
pr_commit_date_0              11/28/1982
pr_term_0                       55 years
pr_begins_0                   12/10/1982
pr_crime_1                       No_data
pr_commit_date_1                 No_data
pr_term_1                        No_data
pr_begins_1                      No_data
pr_crime_2                       No_data
pr_commit_date_2                 No_data
pr_term_2                        No_data
pr_begins_2                      No_data
pr_crime_3                       No_data
pr_commit_date_3                 No_data
pr_term_3                        No_data
pr_begins_3                      No_data
sex                                 Male
race                               Black
age                                   62
max_sentence                    55 years
prison_unit                       Allred
DOB                           10/22/1956
home_county     

In [66]:
def find_crime(row):
    if row['pr_crime_3'] != 'No_data':
        return row['pr_crime_3']
    
    elif row['pr_crime_2'] != 'No_data':
        
        return row['pr_crime_2']
    
    elif row['pr_crime_1'] != 'No_data':
        return row['pr_crime_1']
    
    else:
        return row['pr_crime_0']

In [67]:
df['feature_crime'] = df.apply(find_crime, axis=1)

In [68]:
#checking == success
df[['name', 'pr_crime_0', 'pr_crime_1', 'pr_crime_2', 'pr_crime_3', 'feature_crime']][543:547]

,name,pr_crime_0,pr_crime_1,pr_crime_2,pr_crime_3,feature_crime
543,Jose Alonso Galindo,AGG ASLT PUB SERV/DW,No_data,No_data,No_data,AGG ASLT PUB SERV/DW
544,Litter Allen Ford,AGG ROBBERY,AGG ROBBERY,No_data,No_data,AGG ROBBERY
545,Brian Anthony Gamero,INTOX MANSLAUGHTER W/VEH,INTOX ASSLT W/VEH SBI,POSS CONT SUB 1-4G,No_data,POSS CONT SUB 1-4G
546,Samuel Hendricks,AGG ROBBERY,AGG ROBBERY,AGG ROBBERY,ROBBERY,ROBBERY


In [69]:
def find_start_date(row):
    if row['pr_begins_3'] != 'No_data':
        return row['pr_begins_3']
    
    elif row['pr_begins_2'] != 'No_data':
        
        return row['pr_begins_2']
    
    elif row['pr_begins_1'] != 'No_data':
        return row['pr_begins_1']
    
    else:
        return row['pr_begins_0']

In [70]:
df['feature_startdate'] = df.apply(find_start_date, axis=1)

In [71]:
def find_term(row):
    if row['pr_term_3'] != 'No_data':
        return row['pr_term_3']
    
    elif row['pr_term_2'] != 'No_data':
        
        return row['pr_term_2']
    
    elif row['pr_term_1'] != 'No_data':
        return row['pr_term_1']
    
    else:
        return row['pr_term_0']

In [72]:
#i might make this a feature, no? 
df['feature_term'] = df.apply(find_term, axis= 1)

In [73]:
def find_commit_date(row):
    if row['pr_commit_date_3'] != 'No_data':
        return row['pr_commit_date_3']
    
    elif row['pr_commit_date_2'] != 'No_data':
        
        return row['pr_commit_date_2']
    
    elif row['pr_commit_date_1'] != 'No_data':
        return row['pr_commit_date_1']
    
    else:
        return row['pr_commit_date_0']

In [74]:
df['feature_commit_date'] = df.apply(find_commit_date, axis = 1)

Creating the `target_value` column.

In [75]:
df.loc[5]

name                     Damicio A Sanchez
TDCJ_ID                             701616
pr_crime_0               INDECENCY W/CHILD
pr_commit_date_0                 4/10/1994
pr_term_0                         55 years
pr_begins_0                      4/10/1994
pr_crime_1                         No_data
pr_commit_date_1                   No_data
pr_term_1                          No_data
pr_begins_1                        No_data
pr_crime_2                         No_data
pr_commit_date_2                   No_data
pr_term_2                          No_data
pr_begins_2                        No_data
pr_crime_3                         No_data
pr_commit_date_3                   No_data
pr_term_3                          No_data
pr_begins_3                        No_data
sex                                   Male
race                              Hispanic
age                                     79
max_sentence                      55 years
prison_unit                         Allred
DOB        

In [76]:
#checking each for the most updated version
#i either made a mistake before (prob) or the df is in a diff order this time 
print(df.loc[5][14]) #pc_3
print(df.loc[5][10]) #pc_2
print(df.loc[5][6]) #pc_1
print(df.loc[5][2]) #pc_0 (aka current)

No_data
No_data
No_data
INDECENCY W/CHILD


In [77]:
#need to check this each time as I adjust the filter release range to confirm
target_list = []

for i in range(0, df.shape[0]):
    pc_3 = df.loc[i][14]
    pc_2 = df.loc[i][10]
    pc_1 = df.loc[i][6]
    pc_0 = df.loc[i][2]
    
    if pc_3 != 'No_data':
        target_list.append(pc_2)
        
    elif pc_2 != 'No_data':
        target_list.append(pc_1)
    
    elif pc_1 != 'No_data':
        target_list.append(pc_0)
    
    elif pc_0 != 'No_data':
        target_list.append(0) #this means current is only, no reoffending at this time

In [78]:
#HAVE TO CHECK THIS - IT WAS OFF
target_list[:10]

['HARASSMENT BY PER/CORR FACIL',
 'POSS DDLY WPM PENAL INSTI',
 0,
 0,
 0,
 0,
 0,
 'CONSPIRACY TO COMM. MURDER',
 0,
 0]

In [79]:
df.shape[0]

40828

In [80]:
len(target_list)

40828

In [81]:
df['target_value'] = target_list

In [82]:
df['target_value'][:5]

0    HARASSMENT BY PER/CORR FACIL
1       POSS DDLY WPM PENAL INSTI
2                               0
3                               0
4                               0
Name: target_value, dtype: object

In [83]:
def target(cell):
    if cell != 0:
        return 1
    else:
        return cell

In [84]:
df['final_target'] = df['target_value'].apply(target)

In [85]:
df[:5]

,name,TDCJ_ID,pr_crime_0,pr_commit_date_0,pr_term_0,pr_begins_0,pr_crime_1,pr_commit_date_1,pr_term_1,pr_begins_1,pr_crime_2,pr_commit_date_2,pr_term_2,pr_begins_2,pr_crime_3,pr_commit_date_3,pr_term_3,pr_begins_3,sex,race,age,max_sentence,prison_unit,DOB,home_county,proj_release_date,year_of_release,feature_crime,feature_startdate,feature_term,feature_commit_date,target_value,final_target
0,Victor Cantu,327167,HARASSMENT BY PER/CORR FACIL,2/20/2003,9 years,2/20/2003,HARASSMENT BY PER/CORR FACIL,2/20/2003,9 years,2/20/2003,MURDER W/DEADLY WPN,3/3/1980,60 years,9/7/1980,No_data,No_data,No_data,No_data,Male,Hispanic,71,60 years,Allred,3/2/1948,Hockley,2044-11-23,2044,MURDER W/DEADLY WPN,9/7/1980,60 years,3/3/1980,HARASSMENT BY PER/CORR FACIL,1
1,Pascual Gutierrez Salinas,394125,POSS DDLY WPM PENAL INSTI,7/19/1997,5 years,2/10/1998,MURDER,8/6/1984,60 years,8/6/1984,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,Hispanic,62,60 years,Allred,4/21/1957,Cameron,2047-08-16,2047,MURDER,8/6/1984,60 years,8/6/1984,POSS DDLY WPM PENAL INSTI,1
2,Larry Dewayne Manuel,843835,MURDER,2/2/1994,55 years,2/3/1994,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,Black,42,55 years,Allred,3/12/1977,Dallas,2049-02-02,2049,MURDER,2/3/1994,55 years,2/2/1994,0,0
3,Wayne Edward Cole,358223,BURG OF HABITATION,11/28/1982,55 years,12/10/1982,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,Black,62,55 years,Allred,10/22/1956,Panola,2037-12-10,2037,BURG OF HABITATION,12/10/1982,55 years,11/28/1982,0,0
4,Alberto Rivas,626012,MURDER W/DEADLY WPN,9/22/1991,55 years,9/22/1991,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,No_data,Male,Hispanic,51,55 years,Allred,5/13/1968,Hale,2046-09-22,2046,MURDER W/DEADLY WPN,9/22/1991,55 years,9/22/1991,0,0


In this notebook:
1. The two `.csvs are merged`.
2. The features related to the crime are created (not yet categorized).
3. Anyone having more than 30 years remaining on their projected release date has been filtered out. 
4. The target column has been created. You can see both what they've done just for the visual, and also the `1 or 0`. 

In [86]:
#ALL models, regardless, start with this guy.
df.to_csv('../datasets/my_data/feature_cats.csv', index=False)

In the next notebook:
1. Find `age` at the time of feature crime. 
2. Categorize crimes. 
2. Check value counts (`at the end`) and filter out random 1s for the holdout modeling.